# Alles ist relativ. Objektemaße in Relation setzen

Im Folgenden Beispiel sollen die Maße von Objekten aus der [Deutschen Digitalen Bibliothek](https://www.deutsche-digitale-bibliothek.de) in Relation zu einer anderen Bezugsgröße gesetzt werden.
Wir wollen also Nik, eine 1,71 m große Person (deutscher Durchschnitt laut [Wikipedia](https://en.wikipedia.org/wiki/Average_human_height_by_country)), auf unsere Bilder schauen lassen.

Ein Bild von Nik findet sich auf [Wikimedia Commons](https://commons.wikimedia.org/wiki/File:Person_(13091)_-_The_Noun_Project.svg).

![Nik](https://upload.wikimedia.org/wikipedia/commons/thumb/9/97/Person_%2813091%29_-_The_Noun_Project.svg/512px-Person_%2813091%29_-_The_Noun_Project.svg.png)

In [ ]:
# Import der benötigten Module
import requests
from PIL import Image, ImageDraw, ImageFont
from io import BytesIO
from lxml import etree
from ddbAPI_helpers import search2API, iterateAPICall, getItem

In [ ]:
# Download des Bildes 
nik_img = requests.get("https://upload.wikimedia.org/wikipedia/commons/thumb/9/97/Person_%2813091%29_-_The_Noun_Project.svg/512px-Person_%2813091%29_-_The_Noun_Project.svg.png")
nik_img = Image.open(BytesIO(nik_img.content)).convert('RGBA')

nik_real_height = 171 # Größe in cm
nik_img_height = nik_img.height # Höhe des Bildes in Pixel

## Download der Bilder aus der Deutschen Digitalen Bibliothek

Als Beispiel sollen die [390 ohne Einschränkungen verwendbaren Gemälde in der DDB](https://www.deutsche-digitale-bibliothek.de/searchresults?isThumbnailFiltered=true&query=&offset=0&facetValues%5B%5D=objecttype_fct%3DGem%C3%A4lde&facetValues%5B%5D=license_group%3Drights_001) dienen.

In [ ]:
api_key = "API-KEY"

In [ ]:
#apiurl = search2API("https://www.deutsche-digitale-bibliothek.de/searchresults?isThumbnailFiltered=true&query=&offset=0&facetValues%5B%5D=objecttype_fct%3DGem%C3%A4lde&facetValues%5B%5D=license_group%3Drights_001", api_key)
apiurl = search2API("https://www.deutsche-digitale-bibliothek.de/searchresults?isThumbnailFiltered=true&query=br%C3%BCcke%20museum&facetValues%5B%5D=objecttype_fct%3DGem%C3%A4lde&offset=0", api_key)

In [ ]:
# Download der Objektmetadaten
paintings = iterateAPICall(apiurl, api_key)

Als nächstes brauchen wir (1) die Maße des Objekts und (2) die URL der Bilddatei.
Beide Informationen sind in der Regel in den LIDO-Metadaten enthalten.
Die Maße finden sich im Element `lido:objectMeasurementsSet`, die Bild-URL unter `lido:resourceRepresentation/lido:linkResource`.

Mit untenstehendem Code werden diese Informationen aus dem LIDO-XML extrahiert.
Bei der Gelegenheit werden auch die object-ID sowie der Objekt-Titel ermittelt, damit sie später mit ausgegeben werden können.

Die Maßzahlen sind meist in Zentimetern angegeben.
Zur Umrechnung der Gleitkommazahlen in 'richtige' `float`-Zahlen schreiben wir eine kleine Funktion `unitsConverter`, die nötigenfalls auch Meter oder Millimeter in Zentimeter umrechnet.

In [ ]:
def unitsConverter(value:str,unit:str) -> float:
    '''
    Wandelt eine Zahlen- und Einheitsangabe in eine Float-Zahl und cm um
    '''
    value = value.replace(',','.')
    value = float(value)
    
    if unit == "mm":
        value = value / 100
    if unit == "m":
        value = value * 100
    
    return value

Jetzt können wir über die Liste der Bilder iterieren und die später benötigten Informationen in einer einer neuen Liste (`list_of_paintings`) ablegen.
Die Elemente der Liste sind 4er-Tupel mit folgenden Elementen:

1. Objekt-ID
2. Objekt-Titel
3. Bild-URL
4. tatsächliche Höhe des Objekts in cm

Da wir nicht immer davon ausgehen können, dass alle LIDO-XML-Dateien die von uns benötigten Elemente enthalten, mussten ein paar `try`-`except`-Anweisungen eingebaut werden.
Ansonsten würde ein fehlendes Element zum Abbruch des Skripts führen.

Verarbeitet werden nur die Bilder, die die benötigten Informationen in vorgesehender Form beinhalten.

In [ ]:
# LIDO-Namespace definieren

NSMAP = {'lido' : "http://www.lido-schema.org"}

In [ ]:
list_of_paintings

In [ ]:
list_of_paintings = []
for p in paintings:
    
    object_height = None
    img_url = None
    
    # Objekt-ID extrahieren und die entsprechend LIDO-Datei laden
    object_ID = p.get('id')
    
    res = getItem(p.get('id'), api_key)
    try:
        lido = res.json().get('source').get('record').get('$')
        lido_parsed = etree.fromstring(bytes(lido, encoding='utf-8'))

        # Objekttitel
        try:
            objectTitle = lido_parsed.find('.//lido:titleWrap/lido:titleSet/lido:appellationValue', NSMAP).text
        except:
            objectTitle = "o.T."

        # Maße ermitteln
        objectMeasurementSet = lido_parsed.find('.//lido:objectMeasurementsWrap/lido:objectMeasurementsSet', NSMAP)
        try:
            for measurementSet in objectMeasurementSet.findall('lido:objectMeasurements/lido:measurementsSet', NSMAP):
                measurementType = measurementSet.find('lido:measurementType', NSMAP)
                try:
                    if measurementType.text == "Höhe":
                        object_height_value = measurementSet.find('lido:measurementValue', NSMAP).text
                        object_height_unit = measurementSet.find('lido:measurementUnit', NSMAP).text

                        object_height = unitsConverter(object_height_value,object_height_unit)

                except Exception as e:
                    print(e)
                    object_height = None

        except Exception as e:
            print(e)

        # Bild-URL ermitteln (etwas unsauber wird hier der Einfachheit halber der erste Bild-Link verwendet,
        # nicht unbedingt derjenige, der zum Bild mit der besten Auflösung führt

        try:
            img_url = lido_parsed.find('.//lido:resourceRepresentation/lido:linkResource', NSMAP).text
        except Exception as e:
            print(e)
            img_url = None

        if object_height is not None and img_url is not None:
            list_of_paintings.append((object_ID,objectTitle,img_url,object_height))
            print(object_ID,objectTitle,img_url,object_height)
    except Exception as e:
        print(e)
        print("Vermutlich kein LIDO.")

## Generierung des Bildes

Wir wollen Nik vor ein Bild stellen.
Dazu benötigen wir zunächst einen weißen Hintergrund (die 'Museumswand'), auf den wir das entsprechend skalierte Objekt setzen.

Dann stellen wir Nik davor.
Die Koordinaten sind so gewählt, 

1. dass 15% der Bildbreite rechts über die Symmettrieachse von Nik hinausragt und
2. dass die Unterseite des Bildes bei 85% von Niks Körpergröße liegt.

Die Bildgenerierung erfolgt mit der Funktion `imgForNik`, die als Argument das oben genierte 4er-Tupel mit Objekt-ID, den Bildtitel, die Bild-URL und die Objekthöhe verlangt.
Diese Informationen hatten wir vorhin in den Tupeln der Liste `list_of_paintings` hinterlegt.

In [ ]:
def imgForNik(elem:tuple) -> Image:
    
    object_ID, title, imgurl, object_height = elem
    
    # Download des Bildes
    img = requests.get(imgurl)
    img = Image.open(BytesIO(img.content))
    
    object_img_height = img.height
    
    print(f"https://www.deutsche-digitale-bibliothek.de/item/{object_ID}")
    
    # Größe des Bildes anpassen
    resize_factor = nik_img_height / nik_real_height * object_height /object_img_height
    
    img = img.resize( [int(resize_factor * s) for s in img.size] ) 
    
    # Erstellen des Hintergrunds für Bild und Nik
    bg_width = 1200
    bg_height = 800
    background = Image.new("RGB", (bg_width, bg_height), (255, 255, 255))
    
    # Objekt aufhängen
    background.paste(img, (bg_width - int( nik_img.width / 2 ) - int(img.width * 0.85), bg_height - int( nik_img.height * 0.85 ) - img.height))
    
    # Nik ins Bild setzen
    background.paste(nik_img,(bg_width - nik_img.width, bg_height - nik_img.height), nik_img)

    # Titel ins Bild setzen
    
    # Titel kürzen, falls länger als n Zeichen
    n = 30
    if len(title) > n:
        title = f"{title[:n]} ..."
        
    textLayer = ImageDraw.Draw(background)
    
    myFont = ImageFont.truetype("Arial.ttf", size = 40)
 
    textLayer.text((28, 36), title, font = myFont, fill=(0,0,0))
    
    display(background)
    return background

Mit der `.choices()`-Methode aus dem `random`-Modul lässt sich nun eine zufällige Auswahl aus der Gemälde-Liste ziehen und entsprechend visualisieren. 
Alternativ kann man natürlich auch über die gesamte Liste iterieren und Nik vor alle Bilder stellen.

In [ ]:
import random

In [ ]:
for painting in random.choices(list_of_paintings, k = len(list_of_paintings)):
    imgForNik(painting)